In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model
from tensorflow import keras

In [2]:
data = pd.read_csv('FULLUwi.csv')

In [ ]:
data.head()

In [4]:
list_features = data.columns
print('They are',len(list_features),'features in the dataset.')
print('----------------')
for f in list_features:
    print('feature:', f, '|| Type:', type(data[f][0]), '|| Example:', data[f][0], '|| number of unique values', len(data[f].unique()) )

They are 14 features in the dataset.
----------------
feature: Subject || Type: <class 'str'> || Example: ACCT || number of unique values 194
feature: CourseCode || Type: <class 'numpy.int64'> || Example: 1005 || number of unique values 1256
feature: CourseType || Type: <class 'str'> || Example: E || number of unique values 17
feature: CourseType1 || Type: <class 'str'> || Example: E11 || number of unique values 477
feature: Semester || Type: <class 'numpy.int64'> || Example: 1 || number of unique values 4
feature: Year || Type: <class 'str'> || Example: 2021/2022 || number of unique values 5
feature: Faculty || Type: <class 'str'> || Example: FSS || number of unique values 36
feature: Level || Type: <class 'str'> || Example: 1 || number of unique values 11
feature: Location || Type: <class 'str'> || Example: Mona - Weekend || number of unique values 18
feature: Lecturer || Type: <class 'str'> || Example: Paul, Dwayney (Primary)  || number of unique values 4946
feature: Students || Typ

In [5]:
data.isnull().sum()

Subject            0
CourseCode         0
CourseType         2
CourseType1        0
Semester           0
Year               0
Faculty          718
Level              0
Location           0
Lecturer        4247
Students           0
Seats              0
Room             718
Attribute      50160
dtype: int64

In [6]:
data['Faculty'].unique()

array(['FSS', 'None', 'MS', 'FMS', 'FHE', 'FST', 'ST', 'FEN', 'HE',
       'TMRI', 'WJC', 'FGS', 'RHQ', 'LAW', 'RM', 'FSP', 'STMT', 'SS',
       'SMRT', 'FO', nan, 'UWISN', 'SKILLS', 'MOH', 'SA', 'PSYC', 'ANNEX',
       'SD', 'MED', 'COMMRM', 'LS', 'BIOCHEM', 'ANAT', 'CH', 'COMP',
       'LECT'], dtype=object)

In [7]:
values = {'CourseType':'None','Faculty':'UN','Lecturer':'UN','Room':'UN','Attribute':'None'}
data= data.fillna(value=values)

In [8]:
column = ['Attribute','Students','Seats']
dataframe = data.loc[:,~data.columns.isin(column)].copy()
dataframe['Faculty'] = pd.Categorical(data['Faculty']).codes
labels = dataframe.pop("Faculty")
feature_space = tf.keras.utils.FeatureSpace(
    features={
        "Subject": "string_categorical",
        "CourseCode": "integer_categorical",
        "CourseType": "string_categorical",
        "CourseType1": "string_categorical",
        "Semester": "integer_categorical",
        "Year": "string_categorical",
        "Level": "string_categorical",
        "Location": "string_categorical",
        "Lecturer": "string_categorical",
        "Room": "string_categorical",
    },
    output_mode="concat",
)
dataset = tf.data.Dataset.from_tensor_slices(dict(dataframe))
# Before you start using the FeatureSpace,
# you must `adapt()` it on some data.
feature_space.adapt(dataset)

# You can call the FeatureSpace on a dict of data (batched or unbatched).
output_vector = feature_space(dict(dataframe))

In [9]:
X_train = output_vector[:40000]
X_test = output_vector[40000:]
y_train = labels[:40000]
y_test = labels[40000:]

In [10]:
tf.random.set_seed(42)
model = keras.models.Sequential([
 keras.layers.Dense(300, kernel_initializer = 'lecun_normal', activation="selu",  input_shape=X_train.shape[1:]),
 keras.layers.Dense(100, kernel_initializer = 'lecun_normal', activation="selu"),
 keras.layers.Dense(36, activation="softmax")
])
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
 restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_model.h5", save_best_only=True)
opt = keras.optimizers.Nadam(learning_rate=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer = 'sgd', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=300,
 validation_data=(X_test, y_test), callbacks=[early_stopping_cb,model_checkpoint_cb])

Epoch 1/300
1250/1250 [==============================] - 29s 22ms/step - loss: 1.1432 - accuracy: 0.6991 - val_loss: 0.6184 - val_accuracy: 0.8725
Epoch 2/300
1250/1250 [==============================] - 27s 22ms/step - loss: 0.4394 - accuracy: 0.9019 - val_loss: 0.3581 - val_accuracy: 0.9364
Epoch 3/300
1250/1250 [==============================] - 28s 22ms/step - loss: 0.2319 - accuracy: 0.9543 - val_loss: 0.2659 - val_accuracy: 0.9588
Epoch 4/300
1250/1250 [==============================] - 28s 22ms/step - loss: 0.1457 - accuracy: 0.9739 - val_loss: 0.2199 - val_accuracy: 0.9647
Epoch 5/300
1250/1250 [==============================] - 28s 22ms/step - loss: 0.1016 - accuracy: 0.9830 - val_loss: 0.1962 - val_accuracy: 0.9673
Epoch 6/300
1250/1250 [==============================] - 28s 22ms/step - loss: 0.0757 - accuracy: 0.9890 - val_loss: 0.1812 - val_accuracy: 0.9714
Epoch 7/300
1250/1250 [==============================] - 28s 22ms/step - loss: 0.0591 - accuracy: 0.9922 - val_loss: 0

In [11]:
prediction = np.argmax(model.predict(output_vector), axis = 1)

1884/1884 [==============================] - 15s 8ms/step


In [12]:
data['PreFaculty'] = prediction
data['PreFaculty'] = data.apply(lambda x: pd.Categorical(data['Faculty']).categories[x.PreFaculty],axis=1)
data.to_csv('FTest.csv',index=False)